In [1]:
!pip install -q oauth2client==1.5.2
import sys
sys.path.append("./utils/student_function/")
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from mooclib import *
import inspect, urllib
html, auth_code, userinfo = google_authenticate(PORT_NUMBER=8080)
html

CLICK ON THIS LINK TO AUTHENTICATE WITH YOUR GMAIL ACCOUNT
https://accounts.google.com/o/oauth2/auth?client_id=486275613361-7nk9ktukfcfjevfoe3tamjp0pmbjmprs.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080&scope=profile+email&access_type=offline&response_type=code
waiting for authentication ...
authentication succeeded


,sebastian.ruiz8@udea.edu.coSEBASTIAN RUIZ PALACIOgoogle id: 114687344014261575746authorization code: 4/rAHj4Wz4bsDQF3vc1bbgjtLL4ZDbAWYYmwETaj5AIc8Z8EBuDJc-2YNwsJXLJJfgvFOUG7vKw2WYgz1NWPKVlqc


# Ejercicio 1

completa al siguiente función para que dada una especificación de un laberinto como en las notas devuelva las coordenadas de la casilla **`T`**. Por ejemplo:

    my_grid = [".....X",
               "XX...X",
               "...X..",
               ".XX..X",
               ".X...T"]
                  
    find_target(my_grid)
    (4,5)
    
observa que en la coordenada resultante el primer elemento es la columna y el segundo la fila.

**el resultado ha de ser del tipo `tupla`**

_sugerencia_: usa la función [`np.argwhere`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.argwhere.html)

In [75]:
a = [1,2,3,4,5]
hl = np.array(a)
range(len(hl))

range(0, 5)

In [81]:
import numpy as np
hola = ["a","b","ab","abc"]
hol = np.array(hola)
for i in range(len(hol)):
    for j in range(len(hol[i])):
        if hola[i][j] == "c":
            tupla = i,j
print(tupla)


(3, 2)


In [92]:
%%writefile ./utils/student_function/PS09_01.py
import numpy as np
def find_target(grid):
    laberinto = np.array(grid)
    for i in range(len(laberinto)):
        for j in range(len(laberinto[i])):
            if laberinto[i][j] == "T":
                target = i,j
                return target


Overwriting ./utils/student_function/PS09_01.py


comprueba tu código

In [93]:
import PS09_01 as st
import importlib
importlib.reload(st)
my_grid = [".....X",
        "XX...X",
        "...X..",
        ".XX..X",
        ".X...T"]

st.find_target(my_grid)

(4, 5)

#### evaluate your answer before submitting:

In [94]:
!./run CHECK_SOLUTION PS09_01 


** CORRECTO **
evaluation result 5.0


#### submit your answer (you must be connected to internet)

In [95]:
!./run SUBMIT_SOLUTION PS09_01 

connecting ... registering submission for sebastian.ruiz8@udea.edu.co ...
no config found for section PS09, allowing all submissions
your submissions sheet is https://docs.google.com/spreadsheets/d/1WaK6Gr9sKYog5B6r3UnFKCNRUOte1AwPbjkDQg5r3kI

** CORRECTO **
----
evaluation result 5.0 , submission registered


# Ejercicio 2

completa la siguiente clase para que implemente una cola con prioridades de forma que:

- el método `put` añada elemnetos asociados a una prioridad
- cada vez que se invoque el método `get` devuelva el elemento con la prioridad **más baja** y lo elimine de la cola.
- si la cola no tiene elementos el método `get` ha de devolver `None`
- el método `empty` ha de devolver `True` si la cola no tiene elementos y `False` si tiene alguno

TypeError: 'builtin_function_or_method' object is not iterable

In [147]:
a = [1,2,3,4,5,1]
a.index(1)

0

In [150]:
%%writefile ./utils/student_function/PS09_02.py
def PriorityQueue():
    class _PriorityQueue:
        def __init__(self):
            self.items = []
            self.priority = []
            
        def empty(self):
            return True if len(self.elements) == 0 else False

        def put(self, item, priority):
            self.items.append(item)
            self.priority.append(priority)

        def get(self):
            menor = []
            return menor
        
    return _PriorityQueue()


Overwriting ./utils/student_function/PS09_02.py


verifica tu código. el siguiente ejemplo debe de recuperar los elementos en el siguiente orden:

        c 
        a
        b 
        None

In [151]:
import PS09_02 as st
import importlib
importlib.reload(st)
q = st.PriorityQueue()

q.put("a", 20)
q.put("b", 30)
q.put("c", 15)

print(q.get())
print(q.get())
print(q.get())
print(q.get())

ValueError: 'c' is not in list

#### evaluate your answer before submitting:

In [ ]:
!./run CHECK_SOLUTION PS09_02 

#### submit your answer (you must be connected to internet)

In [ ]:
!./run SUBMIT_SOLUTION PS09_02 

# Ejercicio 3

observa el algoritmo de búsqueda **`cost_search`** que devuelve dos estructuras:

- **`paths`**: un diccionario en el que, para cada posición, indica desde qué posición anterior se llegó con el menor coste desde la **posición inicial (0,0)**.
- **`costs`**: un diccionario en el que, para cada posición, indica cual es el menor coste de los posibles caminos para llegar a dicha posición.

**Definición**: el coste de un camino es la longitud del mismo

Observa como, desde **`paths`**, se puede recuperar el camino de menor coste a cualquier posición empezando desde un target dado un *caminando hacia atrás*.

P.ej. desde al target (2,4) (esquina inferior derecha), se llega desde el (2,3), y a este desde el (2,2), y a este desde el (1,2), y a este desde el (0,2) y a este desde el (0,1) y a este desde el (0,0). Con lo que el camino completo desde el (0,0) es el siguiente:

    (0,0), (0,1), (0,2), (1,2), (2,2), (2,3), (2,4)
    
con un coste de 6.

In [ ]:
import pandas as pd
import numpy as np
EMPTY, WALL, TARGET, USED = ".","X","T","o"
c  = pd.Series({"EMPTY": ".", "WALL":"X", "TARGET":"T", "USED":"o"})
ci = pd.Series({".": 0, "X": 255, "T":100, "o":200})


def possible_moves(grid, y,x):
    moves = [ [y,x+1], [y-1,x], [y,x-1], [y+1,x]]

    moves = [(my,mx) for my,mx in moves if mx>=0 and mx<len(grid[0]) and \
                                           my>=0 and my<len(grid) and grid[my][mx]!=c.WALL]
    return moves

def cost_search(grid):
    import PS09_02 as st
    start = (0,0)
    step_cost = 1
    frontier = st.PriorityQueue()
    frontier.put(start, 0)
    came_from   = {}
    cost_so_far = {}
    came_from[start]   = None
    cost_so_far[start] = 0

    while not frontier.empty():
        current = frontier.get()
        y,x = current
        if grid[y][x] == c.TARGET:
            break

        for next in possible_moves(grid,y,x):
            new_cost = cost_so_far[current] + step_cost
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost
                frontier.put(next, priority)
                came_from[next] = current

    return came_from, cost_so_far

Completa la función **`recover_path`** más abajo para que dado `paths`, `costs` y una tupla representando una posición `target` devuelva

- el camino especificado en `paths` para llegar a `target`
- el coste del mismo

por ejemplo, con los valores del ejemplo anterior:

    > camino, coste = recover_path(path, cost, (2,4))
    > print camino
    > print coste
    
    [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (2, 3), (2, 4)]
    6

In [ ]:
my_grid = ["....X",
           "XX.XX",
           ".X..T"]

path, cost = cost_search(my_grid)

In [ ]:
path

In [ ]:
cost

### completa esta función

In [ ]:
%%writefile ./utils/student_function/PS09_03.py
def recover_path(path, cost, target):
    # tu cosdigo aqui
    camino = ...
    coste  = ...
    return camino, coste


prueba tu código

In [ ]:
import PS09_03 as st
import importlib
importlib.reload(st)
camino, coste = st.recover_path(path, cost, (2,4))
print(camino)
print(coste)

#### evaluate your answer before submitting:

In [ ]:
!./run CHECK_SOLUTION PS09_03 

#### submit your answer (you must be connected to internet)

In [ ]:
!./run SUBMIT_SOLUTION PS09_03 